# Load Best model and Train Data

Load Best Performing Model

In [14]:
import pickle
# Load the model from the pickle file
with open('best_model.pkl', 'rb') as file:
    model = pickle.load(file)


In [15]:
model

Pipeline(steps=[('pca', PCA(n_components=300)),
                ('svc', SVC(C=15, degree=1, max_iter=700, random_state=42))])

We are going to load the train data as we are going to use the scaler that is fitted on the train data to scale the test data uploaded


In [22]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Load the train data from Mydrive
train_data = pd.read_csv('/content/drive/MyDrive/261_Data/data.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-22-d7dbfd0b8fb9>:7: DtypeWarning: Columns (662,664,676,677,683,685,686,687) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/content/drive/MyDrive/261_Data/data.csv')


In [81]:
import pandas as pd

train_dataframe = pd.DataFrame(train_data)


# Fill Na in train data

In [82]:
import warnings

warnings.filterwarnings('ignore')

# Calculate mean only for numeric columns
numeric_means = train_dataframe.select_dtypes(include=[np.number]).mean()

# Fill NaN values only in numeric columns with their respective mean
train_dataframe.fillna(numeric_means, inplace=True)

# For non-numeric columns, you might want to fill NaNs with a specific value or use another method
# Example: df2['non_numeric_column'].fillna('default_value', inplace=True)

train_dataframe.dropna(inplace=True)

# Count the number of NaN values in each column to verify
print(f'Remaining Null Values =  {train_dataframe.isnull().sum().sum()}')

Remaining Null Values =  0


# **Re-Labeling non-numerical values for train data**



Following the same process in the Code notebook, we relabel the data

Remove irrelevant features

In [83]:
train_dataframe = train_dataframe.drop(columns=['patient_id', 'cohort', 'her2_status', 'er_status'])

Label Categorical columns in-place

In [84]:
from sklearn.preprocessing import LabelEncoder

# Convert set to list
categorical_columns = ["cancer_type", "cellularity", "pam50_+_claudin-low_subtype", "er_status_measured_by_ihc", "her2_status_measured_by_snp6", "inferred_menopausal_state", "primary_tumor_laterality", "pr_status", "3-gene_classifier_subtype", "death_from_cancer"]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for col in categorical_columns:
    train_dataframe[col] = label_encoder.fit_transform(train_dataframe[col])


Handle non-categorical columns

In [85]:
# Convert all non-numeric columns to numeric using one-hot encoding
train_dataframe = pd.get_dummies(train_dataframe)

In [86]:
train_dataframe

,cancer_type,age_at_diagnosis,cellularity,chemotherapy,pam50_+_claudin-low_subtype,er_status_measured_by_ihc,neoplasm_histologic_grade,her2_status_measured_by_snp6,hormone_therapy,inferred_menopausal_state,...,prps2_mut_0,prps2_mut_G255R,prps2_mut_I159F,smarcb1_mut_0,smarcb1_mut_0,smarcb1_mut_D159N,stmn2_mut_0,stmn2_mut_0,siah1_mut_0,siah1_mut_0
0,0,54.29,0,1,3,1,3.000000,2,1,0,...,True,False,False,True,False,False,False,True,True,False
1,0,43.45,2,0,2,1,1.000000,1,1,1,...,True,False,False,True,False,False,False,True,True,False
2,0,74.11,0,0,3,1,3.000000,2,1,0,...,True,False,False,True,False,False,False,True,True,False
3,0,51.87,0,0,2,1,2.000000,2,0,0,...,True,False,False,True,False,False,False,True,True,False
4,0,87.18,2,0,3,1,3.000000,0,1,0,...,True,False,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,3,66.42,2,0,2,1,2.000000,2,0,0,...,True,False,False,False,True,False,True,False,False,True
1489,0,45.76,0,1,2,1,3.000000,2,1,1,...,True,False,False,False,True,False,True,False,False,True
1490,0,57.31,0,0,3,1,3.000000,0,1,0,...,True,False,False,False,True,False,True,False,False,True
1491,0,50.08,2,0,6,1,2.413435,2,1,0,...,True,False,False,False,True,False,True,False,False,True


# **Fit Scaler to train data**

In [87]:

from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler object
scaler = MinMaxScaler()

train_dataframe = train_dataframe.drop('cancer_type', axis=1)

# Fit the scaler to the training data
scaler.fit(train_dataframe)


MinMaxScaler()

# Upload Test Data csv


In [29]:
from google.colab import files

uploaded = files.upload()

Saving data.csv to data.csv


<ipython-input-29-e06db0ba58cd>:7: DtypeWarning: Columns (662,664,676,677,683,685,686,687) have mixed types. Specify dtype option on import or set low_memory=False.
  test_dataframe = pd.read_csv(next(iter(uploaded)))


In [89]:
import pandas as pd

test_dataframe = pd.read_csv(next(iter(uploaded)))

# Fill Na on test data

In [90]:
import warnings

warnings.filterwarnings('ignore')

# Calculate mean only for numeric columns
numeric_means = test_dataframe.select_dtypes(include=[np.number]).mean()

# Fill NaN values only in numeric columns with their respective mean
test_dataframe.fillna(numeric_means, inplace=True)

# For non-numeric columns, you might want to fill NaNs with a specific value or use another method
# Example: df2['non_numeric_column'].fillna('default_value', inplace=True)

test_dataframe.dropna(inplace=True)

# Count the number of NaN values in each column to verify
print(f'Remaining Null Values =  {test_dataframe.isnull().sum().sum()}')

Remaining Null Values =  0


# Re-label test data

In [91]:
test_dataframe = test_dataframe.drop(columns=['patient_id', 'cohort', 'her2_status', 'er_status'])

In [92]:
#Relabel the data uploaded
from sklearn.preprocessing import LabelEncoder
import pandas as pd
# Label Categorical columns in-place

# Convert set to list
categorical_columns = ["cancer_type", "cellularity", "pam50_+_claudin-low_subtype", "er_status_measured_by_ihc", "her2_status_measured_by_snp6", "inferred_menopausal_state", "primary_tumor_laterality", "pr_status", "3-gene_classifier_subtype", "death_from_cancer"]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for col in categorical_columns:
    test_dataframe[col] = label_encoder.fit_transform(test_dataframe[col])

# Handle non-categorical columns
# Convert all non-numeric columns to numeric using one-hot encoding
test_dataframe = pd.get_dummies(test_dataframe)


In [93]:
y_test = test_dataframe['cancer_type']
x_test = test_dataframe.drop('cancer_type', axis=1)


# Scale Test Data

Scale the test data using the scaler fitted on the train

In [94]:
x_test = scaler.transform(x_test)

# Final Prediction and evaluation

Predict the values of x_test

In [95]:
predicted_value = model.predict(x_test)

print out the results of the model predictions

In [96]:
# Display accuracy score, confusion matrix and classification report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Calculate accuracy score
accuracy = accuracy_score(y_test, predicted_value)
print("Accuracy:", accuracy)

# Generate confusion matrix
confusion_matrix = confusion_matrix(y_test, predicted_value)
print("Confusion matrix:")
print(confusion_matrix)

# Generate classification report
classification_report = classification_report(y_test, predicted_value)
print("Classification report:")
print(classification_report)


Accuracy: 0.9190751445086706
Confusion matrix:
[[950   1   0   2]
 [ 38  60   0   0]
 [  8   0   8   0]
 [ 46   3   0  95]]
Classification report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       953
           1       0.94      0.61      0.74        98
           2       1.00      0.50      0.67        16
           3       0.98      0.66      0.79       144

    accuracy                           0.92      1211
   macro avg       0.96      0.69      0.79      1211
weighted avg       0.92      0.92      0.91      1211

